In [ ]:
import numpy as np 
import pandas as pd 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

from data_prep import *
from eda import *

In [ ]:
# Import Data:
df = pd.read_csv("./data/Year 2010-2011.csv")

df.head()

In [ ]:
check_df(df)

In [ ]:
# Categorical / Numerical / Cardinal Features: 

cat_cols, num_cols, cat_but_car = grab_col_names(df)
num_cols = [col for col in num_cols if ("ID" not in col) and ("Date" not in col) ]

In [ ]:
# Let's observe  numerical columns: 

for col in num_cols:
    num_summary(df,col)

In [ ]:
# Missing Values:

missing_values_table(df)

In [ ]:
# Drop NA values:

df.dropna(inplace=True)
missing_values_table(df)

In [ ]:
# Let's remove the returned product transactions (negative values -> Invoice Id contains value "C")

df_Invoice = pd.DataFrame({"Invoice":[row for row in df["Invoice"].values if "C"  not in str(row)]})
df_Invoice.head()
df_Invoice = df_Invoice.drop_duplicates("Invoice")
df = df.merge(df_Invoice, on = "Invoice")

In [ ]:
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]

In [ ]:
# Unique Number of Products (with Description)
df.Description.nunique()

In [ ]:
# Unique Number of Products (with StockCode)
df.StockCode.nunique()

In [ ]:
# The unique values of these 2 variables (Description & StockCode) should be equal, because each stock code represents a product.

df_product = df[["Description","StockCode"]].drop_duplicates()
df_product = df_product.groupby(["Description"]).agg({"StockCode":"count"}).reset_index()

df_product.rename(columns={'StockCode':'StockCode_Count'},inplace=True)
df_product.head()

In [ ]:
df_product = df_product.sort_values("StockCode_Count", ascending=False)
df_product = df_product[df_product["StockCode_Count"]>1]

df_product.head()

In [ ]:
# Let's delete products with more than one stock code 

df = df[~df["Description"].isin(df_product["Description"])]

print(df.StockCode.nunique())
print(df.Description.nunique())

In [ ]:
df_product = df[["Description","StockCode"]].drop_duplicates()
df_product = df_product.groupby(["StockCode"]).agg({"Description":"count"}).reset_index()
df_product.rename(columns={'Description':'Description_Count'},inplace=True)
df_product = df_product.sort_values("Description_Count", ascending=False)
df_product = df_product[df_product["Description_Count"] > 1] 


df_product.head()

In [ ]:
# Let's delete stock codes that represent multiple products
df = df[~df["StockCode"].isin(df_product["StockCode"])]

In [ ]:
# Now each stock code represents a single product

print(df.StockCode.nunique())
print(df.Description.nunique())

In [ ]:
# The post statement in the stock code shows the postage cost, let's delete it as it is not a product
df = df[~df["StockCode"].str.contains("POST", na=False)]

In [ ]:
# Calculating Total Price:
df['TotalPrice'] = df['Quantity'] * df['Price']

In [ ]:
df.head()

# RFM analysis

In [ ]:
df.info()

In [ ]:
# Let's observe the last transaction date.
# So we can determine the performans/measurement date for calculating how recent a customer's latest purchase was.

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['InvoiceDate'].max()

In [ ]:
# Assign "performans_date" as 2 days after the last transaction date of purchase:

performans_date = df["InvoiceDate"].max() + timedelta(days=2)
performans_date

In [ ]:
rfm_df = df.groupby("Customer ID").agg \
                                    ({"InvoiceDate" : lambda InvoiceDate :(performans_date - InvoiceDate.max()).days,  # Recency
                                     "Invoice" : lambda Invoice: Invoice.nunique(),  # Frequency
                                     "TotalPrice":  lambda Total_Price: Total_Price.sum()})    # Monetary

In [ ]:
rfm_df.head()

In [ ]:
# Replace column names with Recency, Frequency and Monetary:

rfm_df.columns = ['recency', 'frequency', 'monetary']

rfm_df.head()

In [ ]:
# Let's check if the values inclued any NaN values:

check_df(rfm_df)

# Assigning RFM Scores

In [ ]:
rfm_df["Recency_Score"]  = pd.qcut(rfm_df['recency'], 5, [5,4,3,2,1])
rfm_df["Frequency_Score"]  = pd.qcut(rfm_df['frequency'].rank(method="first"), 5, [1, 2, 3, 4, 5])
rfm_df["Monetary_Score"]  = pd.qcut(rfm_df['monetary'], 5, [1, 2, 3, 4, 5])

In [ ]:
rfm_df["RFM_SCORE"] = (rfm_df['Recency_Score'].astype(str) +
                    rfm_df['Frequency_Score'].astype(str)+
                    rfm_df['Monetary_Score'].astype(str) )

rfm_df.head() 

In [ ]:
rfm_df['Segment'] = rfm_df['RFM_SCORE']
rfm_df.head()

In [ ]:
seg_map = {
    r'5[4-5][4-5]': 'Champions',
    r'[3-5][4-5][4-5]': 'Loyal customers',
    r'[3-5][2-3][3-5]': 'Potential loyalists',
    r'[3-5][3-5][3-5]': 'Need attention',    
    r'[1-5][3-5][1-5]': 'Normal Customer', 
    r'[1-2][3-5][1-2]': 'Small basket size',
    r'5[1-5][1-5]': 'New customers',
    r'4[1-5][1-2]': 'Promising small money',
    r'4[1-5][3-5]': 'Promising big money',
    r'[1-2][1-2][4-5]': 'Long time big buy',
    r'[2-3][1-3][1-5]': 'At Risk',
    r'[1-2][1-2][1-3]': 'Hibernating',
}


In [ ]:
rfm_df['Segment'] = rfm_df['Segment'].replace(seg_map, regex=True)
rfm_df.reset_index(inplace=True)
rfm_df.head()

In [ ]:
rfm_df.groupby('Segment').agg({"Customer ID":"count"}).sort_values("Customer ID",ascending=False)

In [ ]:
rfm_df.groupby('Segment').agg({"Customer ID":"count"}).sort_values("Customer ID",ascending=False).to_csv("ada.csv")

In [ ]:
import matplotlib.pyplot as plt

colors = ("darkorange", "darkseagreen", "orange", "cyan", "cadetblue", "hotpink", "lightsteelblue", "coral", "mediumaquamarine", "palegoldenrod", "lightcoral")
explodes = [0.1] * 11  # Adjust explosion values as needed

# Assuming rfm_df["Segment"] contains the segment names
rfm_df["Segment"].value_counts(sort=False).plot.pie(colors=colors,
                                                    textprops={'fontsize': 12},
                                                    autopct='%4.1f',
                                                    startangle=90,
                                                    radius=2,
                                                    rotatelabels=True,
                                                    shadow=True,
                                                    explode=explodes)

plt.ylabel("")
plt.show()

In [ ]:
rfm_df.to_csv("RFM_customer_segment.csv")